In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input, Embedding, Dot, Add, Flatten, Dense, Concatenate
from keras.layers import Dropout, BatchNormalization, Activation
from keras.regularizers import l2
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import Input, Embedding, Dot, Add, Flatten, Dense, Activation, Dropout, Concatenate
from keras.optimizers import SGD
import numpy as np

In [2]:
# Classify time of day
def classify_time_of_day(timestamp):
    hour = timestamp.hour
    if 4 <= hour < 12:
        return 1
    elif 12 <= hour < 20:
        return 2
    else:
        return 3

# Load data
metadata_path = '../data/id_metadata.csv'
listening_history_path = '../data/listening_history.csv'
metadata_df = pd.read_csv(metadata_path, delimiter='\t')
df = pd.read_csv(listening_history_path, delimiter='\t')

df['timestamp'] = pd.to_datetime(df['timestamp'])
metadata_df.rename(columns = {'id': 'song'}, inplace = True)
df['time_of_day'] = df['timestamp'].apply(classify_time_of_day)
print(df)

                  user              song           timestamp  time_of_day
0        user_007XIjOr  DaTQ53TUmfP93FSr 2019-02-20 12:28:00            2
1        user_007XIjOr  dGeyvi5WCOjDU7da 2019-02-20 12:35:00            2
2        user_007XIjOr  qUm54NYOjeFhmKYx 2019-02-20 12:48:00            2
3        user_007XIjOr  FtnuMT1DlevSR2n5 2019-02-20 12:52:00            2
4        user_007XIjOr  LHETTZcSZLeaVOGh 2019-02-20 13:09:00            2
...                ...               ...                 ...          ...
5109587  user_zzWscYTy  BBiswLufo26YQCT7 2019-01-10 15:57:00            2
5109588  user_zzWscYTy  5ZHgff3sjETIiedr 2019-01-10 16:21:00            2
5109589  user_zzWscYTy  m4O1iLh6fC43xjRy 2019-01-10 16:48:00            2
5109590  user_zzWscYTy  mvUaP8k67qOFfA65 2019-01-10 21:13:00            3
5109591  user_zzWscYTy  BBiswLufo26YQCT7 2019-01-10 21:16:00            3

[5109592 rows x 4 columns]


In [3]:
# Ensure the 'timestamp' column is datetime type
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Find the latest date in the data
latest_date = df['timestamp'].max()

# Calculate the date 14 days (2 weeks) prior to the latest date
two_weeks_ago = latest_date - pd.Timedelta(days=7)

# Filter the DataFrame to include only the last two weeks of data
df = df[df['timestamp'] >= two_weeks_ago]

print(df)

                  user              song           timestamp  time_of_day
5288     user_02jFGVkG  ke5JKpLa8Dw7dCDO 2019-03-19 12:39:00            2
5289     user_02jFGVkG  nRpEFcFzYZ8Z8Cye 2019-03-19 12:44:00            2
5290     user_02jFGVkG  UTDxdZ1outySsU7O 2019-03-19 15:18:00            2
5291     user_02jFGVkG  UTDxdZ1outySsU7O 2019-03-19 15:24:00            2
5292     user_02jFGVkG  bxo3drSzBGDlrodp 2019-03-19 15:24:00            2
...                ...               ...                 ...          ...
5104162  user_zwQunLVn  A8GhNkPLuKusMIvF 2019-03-21 18:29:00            2
5104163  user_zwQunLVn  0m1MyuwLx1EjuQqD 2019-03-21 22:29:00            3
5104164  user_zwQunLVn  0m1MyuwLx1EjuQqD 2019-03-21 22:33:00            3
5104165  user_zwQunLVn  l2kISVY4j8iz1Gg7 2019-03-21 22:36:00            3
5104166  user_zwQunLVn  l2kISVY4j8iz1Gg7 2019-03-21 22:40:00            3

[68090 rows x 4 columns]


In [4]:
df = pd.merge(df, metadata_df[['song', 'release', 'popularity', 'danceability', 'energy', 'key', 'mode', 'valence', 'tempo']], on='song')
numeric_cols_df = df.select_dtypes(include=np.number).columns
sscaler = StandardScaler()
df[numeric_cols_df] = sscaler.fit_transform(df[numeric_cols_df])
print(df)



                user              song           timestamp  time_of_day  \
0      user_02jFGVkG  ke5JKpLa8Dw7dCDO 2019-03-19 12:39:00    -0.142363   
1      user_2uotd4qv  ke5JKpLa8Dw7dCDO 2019-03-19 21:49:00     1.212425   
2      user_McRU0r8i  ke5JKpLa8Dw7dCDO 2019-03-20 18:59:00    -0.142363   
3      user_McRU0r8i  ke5JKpLa8Dw7dCDO 2019-03-20 19:41:00    -0.142363   
4      user_02jFGVkG  nRpEFcFzYZ8Z8Cye 2019-03-19 12:44:00    -0.142363   
...              ...               ...                 ...          ...   
68085  user_zwQunLVn  VnF0TlGM24ZQaFxu 2019-03-21 11:02:00    -1.497152   
68086  user_zwQunLVn  7ViolZVDtOxSjByn 2019-03-21 12:30:00    -0.142363   
68087  user_zwQunLVn  Bk8GOanz19kxdO59 2019-03-21 18:02:00    -0.142363   
68088  user_zwQunLVn  qUXY2aw3nHQ4Qslh 2019-03-21 18:06:00    -0.142363   
68089  user_zwQunLVn  A8GhNkPLuKusMIvF 2019-03-21 18:29:00    -0.142363   

        release  popularity  danceability    energy       key      mode  \
0     -1.482494   -0.369

In [5]:
print(df.isnull().sum())

user            0
song            0
timestamp       0
time_of_day     0
release         0
popularity      0
danceability    0
energy          0
key             0
mode            0
valence         0
tempo           0
dtype: int64


In [6]:
unique_names_song = df.song.unique()
unique_names_user = df.user.unique()

print("unique_names_song:", unique_names_song.shape)
print("unique_names_user:", unique_names_user.shape)

unique_names_song: (18193,)
unique_names_user: (633,)


In [7]:
df.isnull().values.any()# Calculate song popularity
song_popularity = df['song'].value_counts() / len(unique_names_song)
df['song_popularity'] = df['song'].map(song_popularity)

# Create an empty interaction matrix
interaction_matrix = np.zeros((df['user'].nunique(), len(unique_names_song)))

# Map users and songs to matrix indices
user_indices = {user: idx for idx, user in enumerate(df['user'].unique())}
song_indices = {song: idx for idx, song in enumerate(unique_names_song)}

for index, row in df.iterrows():
    user_idx = user_indices[row['user']]
    song_idx = song_indices[row['song']]
    interaction_matrix[user_idx, song_idx] = np.log(row['song_popularity'] + 1)

# Prepare dictionaries to map song IDs to their features
song_features = {
    'release': df.set_index('song')['release'].to_dict(),
    'popularity': df.set_index('song')['popularity'].to_dict(),
    'danceability': df.set_index('song')['danceability'].to_dict(),
    'energy': df.set_index('song')['energy'].to_dict(),
    'key': df.set_index('song')['key'].to_dict(),
    'mode': df.set_index('song')['mode'].to_dict(),
    'valence': df.set_index('song')['valence'].to_dict(),
    'tempo': df.set_index('song')['tempo'].to_dict(),
    'time_of_day': df.set_index('song')['time_of_day'].to_dict(),
}

# Create lists for DataFrame including additional features
user_ids, song_ids, releases, popularities, danceabilities, energies, keys, modes, valences, tempos, interactions, time_of_days = [], [], [], [], [], [], [], [], [], [], [], []
for user in user_indices:
    for song in song_indices:
        user_ids.append(user_indices[user])
        song_ids.append(song_indices[song])
        interactions.append(interaction_matrix[user_indices[user], song_indices[song]])
        # Map each song to its additional features
        releases.append(song_features['release'][song])
        popularities.append(song_features['popularity'][song])
        danceabilities.append(song_features['danceability'][song])
        energies.append(song_features['energy'][song])
        keys.append(song_features['key'][song])
        modes.append(song_features['mode'][song])
        valences.append(song_features['valence'][song])
        tempos.append(song_features['tempo'][song])
        time_of_days.append(song_features['time_of_day'][song])

# Create the interaction DataFrame
interaction_df = pd.DataFrame({
    'user_id': user_ids,
    'song_id': song_ids,
    'release': releases,
    'popularity': popularities,
    'danceability': danceabilities,
    'energy': energies,
    'key': keys,
    'mode': modes,
    'valence': valences,
    'tempo': tempos,
    'time_of_day': time_of_days,
    'interaction': interactions
})

print(interaction_df)

          user_id  song_id   release  popularity  danceability    energy  \
0               0        0 -1.482494   -0.369772     -1.819800 -0.602781   
1               0        1 -1.482494    0.239512      0.021888  0.787531   
2               0        2  0.534436   -0.796271      0.505564 -0.529349   
3               0        3 -1.482494   -0.918128      0.629583 -0.940568   
4               0        4 -1.482494   -0.857200      1.038847 -0.862241   
...           ...      ...       ...         ...           ...       ...   
11516164      632    18188  0.630480   -0.065130     -0.226151 -0.940568   
11516165      632    18189  0.630480   -0.186987      0.567573  1.355406   
11516166      632    18190  0.630480   -0.369772      0.424951  1.272183   
11516167      632    18191  0.342347   -0.369772     -0.288160  0.748368   
11516168      632    18192  0.534436   -0.613486      0.641985  1.071469   

               key      mode   valence     tempo  time_of_day  interaction  
0        -

In [8]:
user_encoder = LabelEncoder()
song_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['user'])
df['song_id'] = song_encoder.fit_transform(df['song'])

N = df.user_id.nunique() # Number of users
M = df.song_id.nunique() # Number of songs

print("Number of users: ", N)
print("Number of songs: ", M)
df.shape, interaction_df.shape

Number of users:  633
Number of songs:  18193


((68090, 15), (11516169, 12))

In [9]:
df_train, df_test = train_test_split(interaction_df, test_size=0.2, random_state=42)
print(df_train, df_test)

          user_id  song_id   release  popularity  danceability    energy  \
3437593       188    17309  0.438392    0.300440      0.046692 -0.475499   
10363724      569    11907  0.726525    0.483225      1.441910 -0.235621   
6280692       345     4107 -3.979646   -1.649269      0.505564  0.860963   
3531948       194     2506 -0.329963    0.178583     -0.219950  0.038525   
4391169       241     6656  0.534436   -0.857200     -0.157940  0.346939   
...           ...      ...       ...         ...           ...       ...   
2234489       122    14943 -1.386450   -0.796271     -0.021519 -0.759436   
4304572       236    11024 -0.329963   -0.735343      0.344338  1.174273   
10081351      554     2429  0.438392   -0.186987      0.914828  0.581922   
6550634       360     1154  0.726525   -0.186987      0.697794 -0.612572   
6423388       353     1259  0.726525    0.361368     -0.933061  0.082584   

               key      mode   valence     tempo  time_of_day  interaction  
3437593   

In [10]:
print("unique interactions: ", df_train.interaction.nunique())

unique interactions:  109


In [11]:
continuous_data_train = df_train.iloc[:,2:-1]
continuous_data_test = df_test.iloc[:,2:-1]
continuous_data_train.shape, continuous_data_test.shape, df_train.shape
print(continuous_data_train)

           release  popularity  danceability    energy       key      mode  \
3437593   0.438392    0.300440      0.046692 -0.475499  1.041305  0.799859   
10363724  0.726525    0.483225      1.441910 -0.235621  1.600552  0.799859   
6280692  -3.979646   -1.649269      0.505564  0.860963 -1.475306  0.799859   
3531948  -0.329963    0.178583     -0.219950  0.038525  0.482058  0.799859   
4391169   0.534436   -0.857200     -0.157940  0.346939 -0.356813  0.799859   
...            ...         ...           ...       ...       ...       ...   
2234489  -1.386450   -0.796271     -0.021519 -0.759436 -0.356813  0.799859   
4304572  -0.329963   -0.735343      0.344338  1.174273 -0.356813 -1.250220   
10081351  0.438392   -0.186987      0.914828  0.581922 -1.475306  0.799859   
6550634   0.726525   -0.186987      0.697794 -0.612572  1.041305  0.799859   
6423388   0.726525    0.361368     -0.933061  0.082584 -0.077189  0.799859   

           valence     tempo  time_of_day  
3437593  -0.208389 

In [12]:
K = 15 # define the size of embeddings, capture the relations in data (10-50)

mu = df_train.interaction.mean()  # Mean interaction for normalization
epochs = 1
model_path = '../helpers/model.h5'
if not os.path.exists(model_path):
    u = Input(shape=(1,))
    s = Input(shape=(1,))
    u_embedding = Embedding(N, K)(u) # (N, 1, K)
    s_embedding = Embedding(M, K)(s) # (N, 1, K)


    ##### main branch
    u_bias = Embedding(N, 1)(u) # (N, 1, 1)
    s_bias = Embedding(M, 1)(s) # (N, 1, 1)
    x = Dot(axes=2)([u_embedding, s_embedding]) # (N, 1, 1)
    x = Add()([x, u_bias, s_bias])
    x = Flatten()(x) # (N, 1)

    # CONTINUOUS BRANCH
    continuous_input = Input(shape=(continuous_data_train.shape[1],))


    ##### side branch
    u_embedding = Flatten()(u_embedding) # (N, K)
    s_embedding = Flatten()(s_embedding) # (N, K)
    y = Concatenate()([u_embedding, s_embedding, continuous_input]) # (N, 2K)
    y = Dense(512)(y)
    y = Activation('elu')(y)
    y = Dropout(0.3)(y)
    y = Dense(512)(y)
    y = Activation('elu')(y)
    y = Dropout(0.3)(y)
    y = Dense(512)(y)
    y = Activation('elu')(y)
    y = Dropout(0.3)(y)
    y = Dense(1)(y)


    ##### merge
    x = Add()([x, y])

    model = Model(inputs=[u, s, continuous_input], outputs=x)
    model.compile(
      loss='mse',
      # optimizer='adam',
      #optimizer=Adam(lr=0.001),
      optimizer=SGD(learning_rate=0.08, momentum=0.9),
      metrics=['mse'],
    )

    # Now, train the model
    r = model.fit(
      x=[df_train.user_id.values, df_train.song_id.values, continuous_data_train.values],
      y=df_train.interaction.values - mu,
      epochs=epochs,
      batch_size=128,
      validation_data=(
        [df_test.user_id.values, df_test.song_id.values, continuous_data_test.values],
        df_test.interaction.values - mu
      )
    )

    plt.plot(r.history['loss'], label="train loss")
    plt.plot(r.history['val_loss'], label="test loss")
    plt.legend()
    plt.show()

    # plot mse
    plt.plot(r.history['mse'], label="train mse")
    plt.plot(r.history['val_mse'], label="test mse")
    plt.legend()
    plt.show()
    model.save(model_path)
    print("Model trained and saved as 'model.h5'.")
else:
    print("Model 'model.h5' already exists. Loading model...")
    # Load and use the existing model
    model = load_model(model_path)

Model 'model.h5' already exists. Loading model...


In [13]:
model.summary()
# Plot training and validation loss and MSE
if 'r' in locals():  # Check if training results are available
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(r.history['loss'], label="train loss")
    plt.plot(r.history['val_loss'], label="test loss")
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(r.history['mse'], label="train mse")
    plt.plot(r.history['val_mse'], label="test mse")
    plt.title('Mean Squared Error')
    plt.xlabel('Epochs')
    plt.ylabel('MSE')
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("No training session to plot, model was loaded from file.")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 15)        9495        ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 15)        272895      ['input_2[0][0]']                
                                                                                              

In [14]:
import numpy as np

user_ids_test = df_test.user_id.values
song_ids_test = df_test.song_id.values
continuous_features = ["release", "popularity", "danceability", "energy", "key", "mode", "valence", "tempo", "time_of_day"]

# Select a specific user for the prediction
specific_user_id = user_ids_test[0]  # Example: taking the first user in the test set

# Prepare input data for the model
M = interaction_df['song_id'].nunique()  # Total number of unique songs
user_input = np.array([specific_user_id] * M)  # Repeat the user ID for each song
song_input = np.array(range(M))  # Array of all unique song IDs

# Map song IDs to indices in df
song_id_to_index = {id: idx for idx, id in enumerate(interaction_df['song_id'].unique())}

# Prepare continuous data for all songs
continuous_data_input = np.array([interaction_df.loc[song_id_to_index[song_id], continuous_features] for song_id in song_input])

# Make predictions for this user with all songs, including the continuous data
predicted_interactions = model.predict([user_input, song_input, continuous_data_input])


# Convert predictions back to the original scale, if needed
mu = df_train.interaction.mean()  # Mean interaction value for normalization (if used during training)
predicted_interactions = predicted_interactions.flatten() + mu

# Determine the number of top recommendations, e.g., top 10
N = 10
top_n_indices = np.argsort(predicted_interactions)[::-1][:N]

# Convert the indices to original song IDs
top_n_song_ids = song_encoder.inverse_transform(top_n_indices)

# Output the recommended songs
print(f"Top {N} recommended song IDs for user {specific_user_id} are:", top_n_song_ids)


569/569 [==============================] - 1s 1ms/step
Top 10 recommended song IDs for user 277 are: ['yDPIzYrb35kO30gd' '5pMLWikvbUlxbw3C' 'VgCslSDAzOerZhTp'
 'BBiswLufo26YQCT7' 'VFz60KPCOCrY9TNy' 'I0xepbeRWgWpFSyH'
 'rnmOxlvA87MrxnHZ' 'UJj71KVEwKdqGsYr' 'dlQMazvCZtdUc4gp'
 '0GohjZ4zTigF1LeU']


In [15]:
print("user_input.shape: ", user_input.shape)
print("continuous_data_input.shape: ", continuous_data_input.shape)

user_input.shape:  (18193,)
continuous_data_input.shape:  (18193, 9)


In [16]:
id_information = pd.read_csv('../data/id_information.csv', sep='\t')
print(id_information)

                      id                           artist  \
0       0009fFIM1eYThaPg                           Cheryl   
1       0010xmHR6UICBOYT                          Oddisee   
2       002Jyd0vN4HyCpqL                 Blue Öyster Cult   
3       006TYKNjNxWjfKjy                         Rhapsody   
4       007LIJOPQ4Sb98qV                   The Chameleons   
...                  ...                              ...   
109264  zzyyPUs7hC9Nz2e1  Frank Carter & The Rattlesnakes   
109265  zzz0n04uuTUA7fNh                    Justin Bieber   
109266  zzzj3LYaZtYtbzSr                   Regina Spektor   
109267  zzznMjZAKnJJXQSj                         Dua Lipa   
109268  zzzwh2ktIWjsR7xp                      Snow Patrol   

                                  song  \
0                           Rain on Me   
1                       After Thoughts   
2                               ME 262   
3                    Flames of Revenge   
4                            Nostalgia   
...            

In [17]:
unique_songs = df[df.user_id == specific_user_id].song.unique()

# Print a user-friendly message
print(f"The user with ID {specific_user_id} has interacted with {len(unique_songs)} unique songs.")

# Print a subset of the songs for brevity
print(unique_songs)

The user with ID 277 has interacted with 24 unique songs.
['RCGLlYCigkopKd7D' 'wG3YQktIKVlZruUu' 'J4tQCUzKKKj3OkV0'
 'AWLAvZOhu4vLV6RA' 'b5ef6NhN3RBrQXIE' 'SAryWbXDkmkFWW3Z'
 'GQoNBkAIX2obbtPn' 'Ax8Us7NbJqe08R0a' 'Br8x0lv4QnA6pnqC'
 'ilrbbVMFVbnwmdFp' 'rPcvhlfan3sOTDX8' '6WlGmqDauWacBf9D'
 'vZ3hXeIO65MA9fBi' 's7f6VKariF11TTjK' 'aAXwzXQZdbdPxFqT'
 'L6AP1Ype1fRuWBwD' '5yY4kVyuJPdbOegL' '2HgkT8eEkxxEo2Tg'
 'bhOXqQHAGOeJqm6I' 'X8ZpmMJo8zT2QvcT' 'MwpFtrA5qYK4v5tl'
 '6U91Xztu28VOYkmU' 'NsNXlxbPuFnyPv9p' 'OBkwaSaAAVVcBaag']


In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Example song features DataFrame
# Assume df_features is a DataFrame with songs as rows and features as columns

# Calculate similarity
def calculate_similarity(target_song_features, songs_features):
    similarity = cosine_similarity([target_song_features], songs_features)
    return similarity[0]  # similarity[0] because the result is in a 2D array

maxi = -1
best_idx = 0
last_5_song_ids = list()

#REWORKa
last_5_song_ids = ['aPPVq97XeQv8mqsU', 'SSA3WorrB4G8ww60', 'lJKIbZNzpS6IsNgh', 'd8QDyWffh9zwJ4Gs', 'oLHuLrmZyV1oEfdu']
#print(last_5_song_ids)


for i in range(10):
    recommended_song_id = top_n_song_ids[i]
    # Get the feature vector for the recommended song
    recommended_song_features = interaction_df.loc[interaction_df.song_id[song_indices[recommended_song_id]]]

    # Get the feature vectors for the last 5 played songs
    last_5_songs_features = np.array([interaction_df.loc[interaction_df.song_id[song_indices[song_id]]] for song_id in last_5_song_ids])

    # Calculate similarity
    similarities = calculate_similarity(np.array(recommended_song_features).reshape(1, -1)[0], last_5_songs_features)
    print(np.sum(similarities), end = "\n\n\n")
    
    if np.sum(similarities) > maxi:
        maxi = np.sum(similarities)
        best_idx = i
print()
print()
print(maxi, best_idx)

4.999985323305133


4.999985226451906


4.999986362778417


4.999986066450398


4.999986040665865


4.999981166694702


4.999985515421169


4.999985078380071


4.999985329503232


4.999920069245894




4.999986362778417 2


In [19]:
recommended_song_id = top_n_song_ids[best_idx]
id_information[id_information.id == recommended_song_id]

,id,artist,song,album_name
55951,VgCslSDAzOerZhTp,Pearl Jam,Life Wasted,Pearl Jam


In [20]:
last_5_songs_info = id_information[id_information['id'].isin(last_5_song_ids)]

# Display the information of the last 5 songs
print(last_5_songs_info)

                     id             artist                  song  \
50240  SSA3WorrB4G8ww60          blink-182  All the Small Things   
64094  aPPVq97XeQv8mqsU        Fleet Foxes               Mykonos   
68866  d8QDyWffh9zwJ4Gs           The 1975         Be My Mistake   
83374  lJKIbZNzpS6IsNgh  Foster the People       Pumped Up Kicks   
88650  oLHuLrmZyV1oEfdu           The 1975                  Mine   

                                      album_name  
50240                         Enema Of The State  
64094                 First Collection 2006-2009  
68866  A Brief Inquiry Into Online Relationships  
83374                                    Torches  
88650  A Brief Inquiry Into Online Relationships  


In [29]:
# from sklearn.metrics import precision_score, recall_score, f1_score

# def get_relevant_songs(user_id, df):
#     """
#     Get a list of relevant songs for a given user.
#     This function needs to be adapted based on how relevance is defined in your dataset.
#     """
#     # Example: get songs that the user has interacted with
#     return df[df.user_id == user_id]['song'].unique()

# # Select a user and predict top N songs
# specific_user_id = user_ids_test[0]
# predicted_interactions = model.predict([user_input, song_input, continuous_data_input])
# flattened_arr = predicted_interactions.flatten()

# # Sort the array in descending order
# sorted_indices = np.argsort(flattened_arr)[::-1]
# sorted_arr = flattened_arr[sorted_indices]

# # Get actual relevant songs for the user
# actual_relevant_songs = get_relevant_songs(specific_user_id, df)

# # Convert actual relevant songs and top recommended songs to a binary format
# actual_binary = [1 if song_encoder.inverse_transform([song_id])[0] in actual_relevant_songs else 0 for song_id in song_ids_test]
# predicted_binary = [1 if song_id in sorted_indices[:len(actual_relevant_songs)].tolist() else 0 for song_id in song_ids_test]

# # Calculate precision, recall, and F1 score
# precision = precision_score(actual_binary, predicted_binary)
# recall = recall_score(actual_binary, predicted_binary)
# f1 = f1_score(actual_binary, predicted_binary)

# print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def precision_at_k(actual, predicted, k):
    act_set = set(actual)
    pred_set = set(predicted[:k])
    result = len(act_set & pred_set) / float(k)
    return result

def recall_at_k(actual, predicted, k):
    act_set = set(actual)
    pred_set = set(predicted[:k])
    result = len(act_set & pred_set) / float(len(act_set))
    return result

def mean_average_precision(actual, predicted, k):
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted[:k]):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(actual), k)

def mean_reciprocal_rank(relevant_results):
    for index, score in enumerate(relevant_results):
        if score:
            return 1.0 / (index + 1)
    return 0

def ndcg_at_k(actual, predicted, k):
    def dcg_at_k(scores, k):
        return sum([score / np.log2(idx + 2) for idx, score in enumerate(scores[:k])])

    actual_scores = [1 if song in actual else 0 for song in predicted[:k]]
    best_scores = sorted(actual_scores, reverse=True)
    ideal_dcg = dcg_at_k(best_scores, k)
    actual_dcg = dcg_at_k(actual_scores, k)
    if not ideal_dcg:
        return 0.0
    return actual_dcg / ideal_dcg

def get_relevant_songs(user_id, df):
    """
    Get a list of relevant songs for a given user.
    This function needs to be adapted based on how relevance is defined in your dataset.
    """
    return df[df.user_id == user_id]['song'].unique()

# Select a user and predict top N songs
specific_user_id = user_ids_test[0]
predicted_interactions = model.predict([user_input, song_input, continuous_data_input])
flattened_arr = predicted_interactions.flatten()

# Sort the array in descending order
sorted_indices = np.argsort(flattened_arr)[::-1]
sorted_arr = flattened_arr[sorted_indices]

# Get actual relevant songs for the user
actual_relevant_songs = get_relevant_songs(specific_user_id, df)

# Convert actual relevant songs and top recommended songs to a binary format
actual_binary = [1 if song_encoder.inverse_transform([song_id])[0] in actual_relevant_songs else 0 for song_id in song_ids_test]
predicted_binary = [1 if song_id in sorted_indices[:len(actual_relevant_songs)].tolist() else 0 for song_id in song_ids_test]

# Calculate precision, recall, and F1 score
precision = precision_score(actual_binary, predicted_binary)
recall = recall_score(actual_binary, predicted_binary)
f1 = f1_score(actual_binary, predicted_binary)

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Top-k calculations
k = 10  # Set this based on your analysis needs
precision_k = precision_at_k(actual_relevant_songs, sorted_indices, k)
recall_k = recall_at_k(actual_relevant_songs, sorted_indices, k)
map_k = mean_average_precision(actual_relevant_songs, sorted_indices, k)
mrr = mean_reciprocal_rank([1 if song_id in actual_relevant_songs else 0 for song_id in sorted_indices])
ndcg_k = ndcg_at_k(actual_relevant_songs, sorted_indices, k)

# Output the additional metrics
print(f"Precision@{k}: {precision_k:.4f}")
print(f"Recall@{k}: {recall_k:.4f}")
print(f"MAP@{k}: {map_k:.4f}")
print(f"MRR: {mrr:.4f}")
print(f"NDCG@{k}: {ndcg_k:.4f}")


569/569 [==============================] - 1s 1ms/step
Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Precision@10: 0.0000
Recall@10: 0.0000
MAP@10: 0.0000
MRR: 0.0000
NDCG@10: 0.0000


In [22]:
# relevant = list()
# for song_id in actual_relevant_songs:
#     relevant.append(song_indices[song_id])
# relevant

In [23]:
# # Placeholder for storing predictions for all users
# predictions = {}
# actual = {}

# # Get predictions and actual relevant songs for each user
# for user_id in user_ids_test[:10]:  # Limit to first 10 users for the example
#     # Predict top N songs
#     user_input = np.array([user_id] * M)
#     continuous_data_input = np.array([interaction_df.loc[song_id_to_index[song_id], continuous_features] for song_id in song_input])
#     predicted_interactions = model.predict([user_input, song_input, continuous_data_input]).flatten()
#     top_n_indices = np.argsort(predicted_interactions)[::-1][:N]
#     top_n_song_ids = song_encoder.inverse_transform(top_n_indices)
    
#     # Store the top N song IDs for each user
#     predictions[user_id] = top_n_song_ids.tolist()
    
#     # Get actual relevant songs for the user
#     actual_relevant_songs = get_relevant_songs(user_id, df)
#     actual[user_id] = actual_relevant_songs.tolist()



569/569 [==============================] - 1s 1ms/step


In [24]:
# def precision_at_k(actual, predicted, k):
#     precision_scores = []
#     for user_id in actual:
#         # Initialize true positives count
#         true_positives = 0
#         # Check if user exists in predictions
#         if user_id in predicted and len(predicted[user_id]) >= k:
#             # Count the number of relevant items in the top k predictions
#             true_positives = len(set(predicted[user_id][:k]) & set(actual[user_id]))
#         # Calculate precision for this user
#         precision = true_positives / float(k)
#         precision_scores.append(precision)
#     # Return the average precision at k for all users
#     return sum(precision_scores) / len(precision_scores)

# def recall_at_k(actual, predicted, k):
#     recall_scores = []
#     for user_id in actual:
#         # Initialize true positives count
#         true_positives = 0
#         # Check if user exists in predictions
#         if user_id in predicted:
#             # Count the number of relevant items in the top k predictions
#             true_positives = len(set(predicted[user_id][:k]) & set(actual[user_id]))
#             recall = true_positives / float(len(actual[user_id]))
#         else:
#             # If no predictions for the user, recall is 0
#             recall = 0.0
#         recall_scores.append(recall)
#     # Return the average recall at k for all users
#     return sum(recall_scores) / len(recall_scores)

# def avg_precision_at_k(actual, predicted, k=10):
#     ap_sum = 0
#     for user, true_items in actual.items():
#         pred_items = predicted[user][:k]
#         hits = 0
#         sum_precs = 0
#         for i, p in enumerate(pred_items):
#             if p in true_items:
#                 hits += 1
#                 sum_precs += hits / (i + 1.0)
#         ap_sum += sum_precs / min(len(true_items), k)
#     return ap_sum / len(actual)

# def mean_avg_precision_at_k(actual, predicted, k=10):
#     return avg_precision_at_k(actual, predicted, k)

# def mean_average_precision_at_k(actual, predicted, k=10):
#     AP_sum = 0.0
#     for user_id in actual:
#         if user_id in predicted:
#             pred_items = predicted[user_id][:k]
#             hits = 0
#             sum_precisions = 0
#             for i, p in enumerate(pred_items):
#                 if p in actual[user_id] and p not in pred_items[:i]:
#                     hits += 1
#                     sum_precisions += hits / (i + 1.0)
#             AP_sum += sum_precisions / min(len(actual[user_id]), k)
#     return AP_sum / len(actual)

# def mean_reciprocal_rank(actual, predicted):
#     MRR_sum = 0.0
#     for user_id in actual:
#         if user_id in predicted:
#             pred_items = predicted[user_id]
#             for rank, p in enumerate(pred_items, start=1):
#                 if p in actual[user_id]:
#                     MRR_sum += 1.0 / rank
#                     break
#     return MRR_sum / len(actual)

# def dcg_at_k(relevances, k):
#     relevances = np.asfarray(relevances)[:k]
#     if relevances.size:
#         return np.sum(relevances / np.log2(np.arange(2, relevances.size + 2)))
#     return 0.0

# def ndcg_at_k(actual, predicted, k=10):
#     NDCG_sum = 0.0
#     for user_id in actual:
#         if user_id in predicted:
#             pred_items = predicted[user_id][:k]
#             true_relevances = [1 if item in actual[user_id] else 0 for item in pred_items]
#             ideal_relevances = [1] * len(actual[user_id])
#             NDCG_sum += dcg_at_k(true_relevances, k) / dcg_at_k(ideal_relevances, k)
#     return NDCG_sum / len(actual)

In [25]:

# k = 10  # You can change this value based on how many recommendations you consider
# precision = precision_at_k(actual, predictions, k)
# recall = recall_at_k(actual, predictions, k)
# map_k = mean_average_precision_at_k(actual, predictions, k)
# mrr = mean_reciprocal_rank(actual, predictions)
# ndcg_k = ndcg_at_k(actual, predictions, k)

# print(f"Precision@{k}: {precision:.4f}")
# print(f"Recall@{k}: {recall:.4f}")
# print(f"MAP@{k}: {map_k:.4f}")
# print(f"MRR: {mrr:.4f}")
# print(f"NDCG@{k}: {ndcg_k:.4f}")


Precision@10: 0.0000
Recall@10: 0.0000
MAP@10: 0.0000
MRR: 0.0000
NDCG@10: 0.0000


In [27]:
# def precision_at_k(actual, predicted, k):
#     act_set = set(actual)
#     pred_set = set(predicted[:k])
#     result = len(act_set & pred_set) / float(k)
#     return result

# def recall_at_k(actual, predicted, k):
#     act_set = set(actual)
#     pred_set = set(predicted[:k])
#     result = len(act_set & pred_set) / float(len(act_set))
#     return result

# def mean_average_precision(actual, predicted, k):
#     score = 0.0
#     num_hits = 0.0
#     for i, p in enumerate(predicted[:k]):
#         if p in actual and p not in predicted[:i]:
#             num_hits += 1.0
#             score += num_hits / (i + 1.0)
#     return score / min(len(actual), k)

# def mean_reciprocal_rank(relevant_results):
#     for index, score in enumerate(relevant_results):
#         if score:
#             return 1.0 / (index + 1)
#     return 0

# def ndcg_at_k(actual, predicted, k):
#     def dcg_at_k(scores, k):
#         return sum([score / np.log2(idx + 2) for idx, score in enumerate(scores[:k])])

#     actual_scores = [1 if song in actual else 0 for song in predicted[:k]]
#     best_scores = sorted(actual_scores, reverse=True)
#     ideal_dcg = dcg_at_k(best_scores, k)
#     actual_dcg = dcg_at_k(actual_scores, k)
#     if not ideal_dcg:
#         return 0.0
#     return actual_dcg / ideal_dcg

# # Implement your song prediction and evaluation logic here
# # For example:
# # specific_user_id = user_ids_test[0]
# # predicted_interactions = model.predict([user_input, song_input, continuous_data_input])
# # flattened_arr = predicted_interactions.flatten()

# # Calculate the ranking metrics
# k = 10  # Top-k items
# predicted_top_k = sorted_indices[:k]
# precision = precision_at_k(actual_relevant_songs, predicted_top_k, k)
# recall = recall_at_k(actual_relevant_songs, predicted_top_k, k)
# map_k = mean_average_precision(actual_relevant_songs, predicted_top_k, k)
# mrr = mean_reciprocal_rank([song_id in actual_relevant_songs for song_id in predicted_top_k])
# ndcg_k = ndcg_at_k(actual_relevant_songs, predicted_top_k, k)

# # Output the additional metrics
# print(f"Precision@{k}: {precision:.4f}")
# print(f"Recall@{k}: {recall:.4f}")
# print(f"MAP@{k}: {map_k:.4f}")
# print(f"MRR: {mrr:.4f}")
# print(f"NDCG@{k}: {ndcg_k:.4f}")

Precision@10: 0.0000
Recall@10: 0.0000
MAP@10: 0.0000
MRR: 0.0000
NDCG@10: 0.0000
